Lectura y ETL

In [3]:
# Importacion de librerias
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go

In [4]:
# Lectura de CSV.
file = 'COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries.csv'
df = pd.read_csv(file)



In [5]:
#ETL
# me quedo con las columnas que voy a utilizar
df.drop(df.columns.difference(['state','date','critical_staffing_shortage_today_yes','inpatient_beds','inpatient_beds_used','inpatient_beds_used_covid','staffed_adult_icu_bed_occupancy','staffed_icu_adult_patients_confirmed_and_suspected_covid','staffed_icu_adult_patients_confirmed_covid','staffed_icu_adult_patients_confirmed_covid','total_adult_patients_hospitalized_confirmed_covid','total_pediatric_patients_hospitalized_confirmed_and_suspected_covid','total_staffed_adult_icu_beds','inpatient_beds_utilization','inpatient_beds_utilization_numerator','inpatient_beds_utilization_denominator','percent_of_inpatients_with_covid','percent_of_inpatients_with_covid_numerator','percent_of_inpatients_with_covid_denominator','inpatient_bed_covid_utilization','inpatient_bed_covid_utilization_numerator','inpatient_bed_covid_utilization_denominator','adult_icu_bed_covid_utilization','adult_icu_bed_covid_utilization_numerator','adult_icu_bed_covid_utilization_denominator',' adult_icu_bed_utilization','deaths_covid','all_pediatric_inpatient_bed_occupied','all_pediatric_inpatient_beds','staffed_icu_pediatric','patients_confirmed_covid','staffed_pediatric_icu_bed_occupancy','total_staffed_pediatric_icu_beds']), axis=1, inplace=True)
# ordeno y transfrmo campo date
df['date']=pd.to_datetime(df['date'])
df = df.sort_values('date',ascending=True)

In [6]:
# hago campo total país, para introducción del análisis
# agrupo por fecha
df_TotalPais_used_bed_covid = df.groupby('date',as_index=False)[['inpatient_bed_covid_utilization_numerator','adult_icu_bed_covid_utilization_numerator','deaths_covid']].sum()
# ordeno por fecha
df_TotalPais_used_bed_covid = df_TotalPais_used_bed_covid.sort_values('date',ascending=True)
#cambio nombres
df_TotalPais_used_bed_covid['Camas ocupadas COVID']=df_TotalPais_used_bed_covid['inpatient_bed_covid_utilization_numerator']
df_TotalPais_used_bed_covid['Camas ocupadas COVID ICU']=df_TotalPais_used_bed_covid['adult_icu_bed_covid_utilization_numerator']
df_TotalPais_used_bed_covid['Muertes COVID']=df_TotalPais_used_bed_covid['deaths_covid']

In [7]:
# Grafico introductorio, ocupación de camas, camas ICU y acumulado de muertes a TOTAL PAIS.

fig = px.line(df_TotalPais_used_bed_covid,x = df_TotalPais_used_bed_covid['date'],
                y=['Camas ocupadas COVID','Camas ocupadas COVID ICU','Muertes COVID'], 
                title='Evolución COVID-19 - United States', width=800, height=550,
                labels= {'date':'Fechas','value':'Cantidad'})

fig.add_trace(
    go.Scatter(x=list(df_TotalPais_used_bed_covid.date), y=list(df_TotalPais_used_bed_covid['Camas ocupadas COVID'])
                , name='Cantidad de camas COVID'))
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()



In [8]:
# Grafico exclusivo de muertes COVID

fig = px.line(df_TotalPais_used_bed_covid,x = df_TotalPais_used_bed_covid['date'],
                y=df_TotalPais_used_bed_covid['Muertes COVID'], 
                title='Evolución COVID-19 - United States', width=800, height=550,
                labels= {'date':'Fechas',
                        'inpatient_beds_used_covid':'Camas ocupadas COVID-19'})

fig.add_trace(
    go.Scatter(x=list(df_TotalPais_used_bed_covid.date), y=list(df_TotalPais_used_bed_covid['Muertes COVID']),name = 'Muertes COVID'))
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()



In [9]:

promedio_internacion = 7.18
df_mapa= df.groupby('state',as_index=False)[['inpatient_bed_covid_utilization_numerator']].sum()
df_mapa['inpatient_bed_covid_utilization_numerator']=round(df_mapa['inpatient_bed_covid_utilization_numerator']/promedio_internacion,0)
df_mapa.rename(columns = {'inpatient_bed_covid_utilization_numerator':'Cantidad_Internaciones'}, inplace = True)
dashfig1 = px.choropleth(df_mapa,locations = df_mapa['state'], locationmode='USA-states',scope='usa',color = 'Cantidad_Internaciones')
dashfig1.update_layout(title_text = 'Internaciones de COVID-19 por Estado',geo_scope='usa')

dashfig1.show()

1 - ¿Cuáles fueron los 5 Estados con mayor ocupación hospitalaria por COVID? Criterio de ocupación por cama común. Considere la cantidad de camas ocupadas con pacientes confirmados y tome como referencia los 6 primeros meses del 2020 - recuerde incluir la cifra de infectados en esos meses (acumulativo). 

La ocupación hospitalaria es un número dinámico, pero los pacientes tienen un promedio de 7.18 días de internación, 5.72 Non-ICU, y 12.34 en ICU.
Fuente:'Hospital length of stay among COVID-19-positive patients (USA)',https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8259605/,

In [10]:
### ESTO AL FINAL NO QUEDO
'''
# seleccionar periodo de Lockdown
df_2020EJ = df[(df['date'].dt.date > pd.to_datetime('2020-01-01')) & (df['date'].dt.date < pd.to_datetime('2020-07-01'))].sort_values('date',ascending=True)
# ordenar por cantidad de internados por dia
df_2020EJ = df_2020EJ.sort_values('inpatient_bed_covid_utilization_numerator', ascending=False)
##### eliminar estados duplicados, para que quede solo el día que mas internads tuvo (buscando el máximo por estado)
df_2020EJ_top = df_2020EJ.drop_duplicates(['state'], keep='first').sort_values('inpatient_bed_covid_utilization_numerator', ascending=False).head()
df_EJstates = df_2020EJ_top[['state','inpatient_bed_covid_utilization_numerator']]

df_EJ_acum = df_2020EJ.groupby('state',as_index=False)[['inpatient_bed_covid_utilization_numerator']].sum()

'''

"\n# seleccionar periodo de Lockdown\ndf_2020EJ = df[(df['date'].dt.date > pd.to_datetime('2020-01-01')) & (df['date'].dt.date < pd.to_datetime('2020-07-01'))].sort_values('date',ascending=True)\n# ordenar por cantidad de internados por dia\ndf_2020EJ = df_2020EJ.sort_values('inpatient_bed_covid_utilization_numerator', ascending=False)\n##### eliminar estados duplicados, para que quede solo el día que mas internads tuvo (buscando el máximo por estado)\ndf_2020EJ_top = df_2020EJ.drop_duplicates(['state'], keep='first').sort_values('inpatient_bed_covid_utilization_numerator', ascending=False).head()\ndf_EJstates = df_2020EJ_top[['state','inpatient_bed_covid_utilization_numerator']]\n\ndf_EJ_acum = df_2020EJ.groupby('state',as_index=False)[['inpatient_bed_covid_utilization_numerator']].sum()\n\n"

In [11]:
promedio_internacion = 7.18
# seleccionar periodo de Lockdown
df_2020EJ = df[(df['date'].dt.date > pd.to_datetime('2020-01-01')) & (df['date'].dt.date < pd.to_datetime('2020-07-01'))].sort_values('date',ascending=True)
#agrupar por estado y sumar valores de utilizacion de camas covid
df_EJ_acum = df_2020EJ.groupby('state',as_index=False)[['inpatient_bed_covid_utilization_numerator']].sum()
# dividirlo por el promedio de dias que esta internado cada persona
df_EJ_acum['inpatient_bed_covid_utilization_numerator']=round(df_EJ_acum['inpatient_bed_covid_utilization_numerator']/promedio_internacion,0)
# ordenarlo y seleccionar el top 5
df_EJ_acum=df_EJ_acum.sort_values('inpatient_bed_covid_utilization_numerator', ascending=False)
internaciones_acum_top = df_EJ_acum.head()


C:\Users\Julian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



In [12]:
internaciones_acum_top

,state,inpatient_bed_covid_utilization_numerator
34,NY,91091.0
4,CA,51094.0
9,FL,34578.0
44,TX,33452.0
14,IL,28517.0


In [13]:


 
 
fig = px.bar(internaciones_acum_top, x = "state", y = "inpatient_bed_covid_utilization_numerator",
         title='Máxima cantidad de camas utilizadas',
        labels= {'state':'Estados',
                        'inpatient_bed_covid_utilization_numerator':'Ocupación hospitalaria (camas)'})
fig.show()

2 - Analice la ocupación de camas (Común) por COVID en el Estado de Nueva York durante la cuarentena establecida e indique:

Intervalos de crecimiento y decrecimiento
Puntos críticos (mínimos y máximos)

La cuarentena en NY fue entre 22/03/2020 al 13/06/2020

In [14]:
# seleccion periodos LockDown
df_NY = df[(df['date'].dt.date > pd.to_datetime('2020-03-21')) & (df['date'].dt.date < pd.to_datetime('2020-06-14'))]
df_NY=df_NY[df_NY['state']=='NY'].sort_values('date',ascending=True)


C:\Users\Julian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



In [15]:

fig = px.line(df_NY,x = df_NY['date'],y=df_NY['inpatient_bed_covid_utilization_numerator'], 
                title='Evolución camas COVID - Nueva York durante el Lockdown', width=800, height=550,
                labels= {'date':'Fechas',
                        'inpatient_bed_covid_utilization_numerator':'Camas ocupadas COVID-19'})
#Creo lineas verticales
fig.add_vline(x='2020-04-14', line_width=3, line_dash="dash", line_color="green")             
fig.add_vrect(x0="2020-04-14", x1="2020-06-13", 
              annotation_text="Máximo 14-04-2020, luego comienza descenso de internaciones", annotation_position="top left",
              annotation=dict(font_size=14, font_family="Times New Roman"),
              fillcolor="green", opacity=0.25, line_width=0)        
fig.show()

In [16]:
def minimo(df):
    valor_minimo = min(df_NY['inpatient_bed_covid_utilization_numerator'])
    minimo_val = df.loc[df_NY['inpatient_bed_covid_utilization_numerator'] == valor_minimo]

    return minimo_val

minimo(df_NY)

,state,date,critical_staffing_shortage_today_yes,inpatient_beds,inpatient_beds_used,inpatient_beds_used_covid,staffed_adult_icu_bed_occupancy,staffed_icu_adult_patients_confirmed_and_suspected_covid,staffed_icu_adult_patients_confirmed_covid,total_adult_patients_hospitalized_confirmed_covid,...,inpatient_bed_covid_utilization_numerator,inpatient_bed_covid_utilization_denominator,adult_icu_bed_covid_utilization,adult_icu_bed_covid_utilization_numerator,adult_icu_bed_covid_utilization_denominator,deaths_covid,all_pediatric_inpatient_bed_occupied,all_pediatric_inpatient_beds,staffed_pediatric_icu_bed_occupancy,total_staffed_pediatric_icu_beds
13522,NY,2020-03-22,0,2171.0,1588.0,324.0,NaN,NaN,NaN,NaN,...,324.0,2171.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN


In [17]:
def maximo(df):
    valor_maximo = max(df_NY['inpatient_bed_covid_utilization_numerator'])
    max_val = df.loc[df_NY['inpatient_bed_covid_utilization_numerator'] == valor_maximo]

    return max_val

maximo(df_NY)

,state,date,critical_staffing_shortage_today_yes,inpatient_beds,inpatient_beds_used,inpatient_beds_used_covid,staffed_adult_icu_bed_occupancy,staffed_icu_adult_patients_confirmed_and_suspected_covid,staffed_icu_adult_patients_confirmed_covid,total_adult_patients_hospitalized_confirmed_covid,...,inpatient_bed_covid_utilization_numerator,inpatient_bed_covid_utilization_denominator,adult_icu_bed_covid_utilization,adult_icu_bed_covid_utilization_numerator,adult_icu_bed_covid_utilization_denominator,deaths_covid,all_pediatric_inpatient_bed_occupied,all_pediatric_inpatient_beds,staffed_pediatric_icu_bed_occupancy,total_staffed_pediatric_icu_beds
13809,NY,2020-04-14,0,33409.0,25178.0,14126.0,NaN,NaN,NaN,NaN,...,13791.0,33271.0,NaN,NaN,NaN,439.0,NaN,NaN,NaN,NaN


3 - ¿Cuáles fueron los cinco Estados que más camas UCI -Unidades de Cuidados Intensivos- utilizaron durante el año 2020? La medición debe realizarse en términos absolutos.

In [18]:
df_2020 = df[(df['date']> pd.to_datetime('2020-01-01')) & (df['date'] < pd.to_datetime('2021-01-01'))].sort_values('date',ascending=True)
df_2020_ICU = df_2020.sort_values('adult_icu_bed_covid_utilization_numerator', ascending=False)
df_2020_ICUST = df_2020_ICU.drop_duplicates(['state'], keep='first').sort_values('adult_icu_bed_covid_utilization_numerator', ascending=False).head()
df_2020_ICUST[['state','date','adult_icu_bed_covid_utilization_numerator']]

,state,date,adult_icu_bed_covid_utilization_numerator
41255,CA,2020-12-31,4587.0
39761,TX,2020-12-31,3290.0
9535,FL,2020-07-24,2493.0
9304,PA,2020-12-14,1418.0
34655,NY,2020-12-31,1379.0


In [19]:

df_2020 = df[(df['date']> pd.to_datetime('2020-01-01')) & (df['date'] < pd.to_datetime('2021-01-01'))].sort_values('date',ascending=True)
df_2020_acum = df_2020.groupby('state',as_index=False)[['adult_icu_bed_covid_utilization_numerator']].sum()

promedio_internacion_icu = 12.34
df_2020_acum['adult_icu_bed_covid_utilization_numerator']=round(df_2020_acum['adult_icu_bed_covid_utilization_numerator']/promedio_internacion_icu,0)
df_2020_acum=df_2020_acum.sort_values('adult_icu_bed_covid_utilization_numerator', ascending=False)
internaciones_acum_top = df_2020_acum.head()

In [20]:
internaciones_acum_top

,state,adult_icu_bed_covid_utilization_numerator
44,TX,28509.0
4,CA,22069.0
9,FL,15191.0
14,IL,8285.0
35,OH,8222.0


In [21]:
fig = px.bar(internaciones_acum_top, x = "state", y = internaciones_acum_top["adult_icu_bed_covid_utilization_numerator"],
         title='Cantidad de camas ICU utilizadas',
        labels= {'state':'Estados',
                        'adult_icu_bed_covid_utilization_numerator':'Cant. de camas ICU utilizadas'})
fig.show()

In [22]:
# maxima utilización de ICU en 2020
df_2020_ICU = df_2020.sort_values('adult_icu_bed_covid_utilization_numerator', ascending=False) 
# me quedo con un único valor (y maximo) por Estado
df_2020_ICU_porc = df_2020_ICU.drop_duplicates(['state'], keep='first').sort_values('adult_icu_bed_covid_utilization', ascending=False).head(5)
df_2020_ICU_porc['adult_icu_bed_covid_utilization']=round(df_2020_ICU_porc['adult_icu_bed_covid_utilization']*100,1)

In [23]:
fig = px.bar(df_2020_ICU_porc, x = "state", y = "adult_icu_bed_covid_utilization",
         title='TOP 5: máximo porcentaje (%) de camas ICU utilizadas en 2020',
        labels= {'state':'Estados',
                        'adult_icu_bed_covid_utilization_numerator':'Máximo porcentaje de camas ICU utilizadas'})
fig.show()

4 - ¿Qué cantidad de camas se utilizaron, por Estado, para pacientes pediátricos con COVID durante el 2020?

In [24]:
df_2020['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid'].replace(np.nan,0)

df_2020_acum_ped = df_2020.groupby('state',as_index=False)[['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid']].sum()


df_2020_acum_ped['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid']=round(df_2020_acum_ped['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid']/promedio_internacion,0)
df_2020_acum_ped=df_2020_acum_ped.sort_values('total_pediatric_patients_hospitalized_confirmed_and_suspected_covid', ascending=False)
internaciones_acum_top_ped = df_2020_acum_ped.head()

In [25]:
# top estados con uso de camas pediatricas
fig = px.bar(internaciones_acum_top_ped, x = "state", y = "total_pediatric_patients_hospitalized_confirmed_and_suspected_covid",
         title='TOP 5 Estados con mayor cantidad de internaciones pediatricas',
        labels= {'state':'Estados',
                        'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid':'Cant. internaciones pediátricas'})
fig.show()

In [26]:
df_2020['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid']

69        NaN
13515     NaN
1305      NaN
16956     NaN
21351     NaN
         ... 
44532     0.0
44513     9.0
44346    10.0
42261     2.0
39473     8.0
Name: total_pediatric_patients_hospitalized_confirmed_and_suspected_covid, Length: 16437, dtype: float64

In [27]:
df_2020_PED = df_2020.sort_values('total_pediatric_patients_hospitalized_confirmed_and_suspected_covid', ascending=False)
df_2020_PEDST = df_2020_PED.drop_duplicates(['state'], keep='first').sort_values('total_pediatric_patients_hospitalized_confirmed_and_suspected_covid', ascending=False).head()
df_2020_ICUST[['state','date','total_pediatric_patients_hospitalized_confirmed_and_suspected_covid']].sort_values('total_pediatric_patients_hospitalized_confirmed_and_suspected_covid', ascending=False)


,state,date,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid
9535,FL,2020-07-24,693.0
39761,TX,2020-12-31,174.0
41255,CA,2020-12-31,154.0
34655,NY,2020-12-31,97.0
9304,PA,2020-12-14,60.0


5 - ¿Qué porcentaje de camas UCI corresponden a casos confirmados de COVID-19? Agrupe por Estado.

La mayoria de las camas UCI fueron utilzadas para pacientes con COVID-19 en el período en análisis

In [28]:
df_icu = df.sort_values('adult_icu_bed_covid_utilization', ascending=False) 
df_ICU_ST = df_icu.drop_duplicates(['state'], keep='first').sort_values('adult_icu_bed_covid_utilization', ascending=False)
df_ICU_ST[['state','date','adult_icu_bed_covid_utilization']].sort_values('adult_icu_bed_covid_utilization', ascending=False)
df_ICU_ST['adult_icu_bed_covid_utilization']=round(df_ICU_ST['adult_icu_bed_covid_utilization']*100,1)
df_ICU_ST_mean = df.groupby('state',as_index=False)[['adult_icu_bed_covid_utilization']].mean()
header = ['state','adult_icu_bed_covid_utilization_mean']
df_ICU_ST_mean.columns = header
df_ICU_ST_mean['adult_icu_bed_covid_utilization_mean']=round(df_ICU_ST_mean['adult_icu_bed_covid_utilization_mean']*100,1)
df_ICU_ST = pd.merge(df_ICU_ST,df_ICU_ST_mean,on='state',how='left')
df_ICU_graph=df_ICU_ST[['state','date','adult_icu_bed_covid_utilization','adult_icu_bed_covid_utilization_mean']]

In [29]:
df_ICU_graph

,state,date,adult_icu_bed_covid_utilization,adult_icu_bed_covid_utilization_mean
0,IL,2021-10-12,118.9,15.0
1,WA,2022-01-17,102.5,14.0
2,AR,2020-07-15,80.4,20.0
3,MS,2020-08-05,74.6,22.1
4,ID,2021-09-24,66.2,21.5
5,NM,2020-12-04,61.1,21.1
6,GA,2021-09-12,60.2,21.7
7,AL,2021-08-30,58.7,19.7
8,AZ,2020-07-15,57.8,17.5
9,CA,2021-01-08,55.4,16.9


In [30]:
df_ICU_graph.drop(df_ICU_graph.tail(39).index,inplace=True)

C:\Users\Julian\AppData\Local\Temp\ipykernel_1868\4169537782.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
states=df_ICU_graph['state']

fig = go.Figure(data=[
    go.Bar(name='% máximo camas ICU por COVID', x=states, y=df_ICU_graph['adult_icu_bed_covid_utilization']),
    go.Bar(name='% promedio camas ICU por COVID', x=states, y=df_ICU_graph['adult_icu_bed_covid_utilization_mean'])
])
# Change the bar mode
fig.update_layout(barmode='group',title='Utilización camas UCI (%)')
fig.show()

6 - ¿Cuántas muertes por covid hubo, por Estado, durante el año 2021?

In [32]:
df_2021 = df[(df['date']> pd.to_datetime('2021-01-01')) & (df['date'] < pd.to_datetime('2022-01-01'))].sort_values('date',ascending=True)
df_2021_muertos = df.groupby('state',as_index=False)[['deaths_covid']].sum()
df_2021_muertos = df_2021_muertos.sort_values('deaths_covid', ascending=False)

In [33]:
#df_2021_muertos = df_2021.sort_values('deaths_covid', ascending=False)
#df_2021_muertos = df_2021_muertos.drop_duplicates(['state'], keep='first').sort_values('deaths_covid', ascending=False)
#df_2021_muertos[['state','date','deaths_covid']]

In [34]:
states=df_2021_muertos['state'].head(15)

fig = go.Figure(data=[
    go.Bar(name='% máximo camas ICU por COVID', x=states, y=df_2021_muertos['deaths_covid'])])
fig.update_layout(title='Cantidad de muertos por Estado')

fig.show()

7 - ¿Qué relación presenta la falta de personal médico, con la cantidad de muertes por covid durante el año 2021?

In [35]:
corr_staff_death = df_2021[['critical_staffing_shortage_today_yes','deaths_covid']]

In [36]:
corr_table = df_2021[['critical_staffing_shortage_today_yes','deaths_covid']].corr()


In [37]:
from scipy import stats

In [38]:
correlation_coef, p_value = stats.pearsonr(df_2021['critical_staffing_shortage_today_yes'],df_2021['deaths_covid'])

In [39]:
print(f'El coeficiente de correlación entre la falta de staff médico y muertes por covid es de {round(correlation_coef,2)}.')
print(f'El coeficiente p_value entre la falta de staff médico y muertes por covid es de {p_value}.' )

El coeficiente de correlación entre la falta de staff médico y muertes por covid es de 0.57.
El coeficiente p_value entre la falta de staff médico y muertes por covid es de 0.0.


!['Correlación'](correlation.png)

In [40]:
# normalizacion de data con método max/min

corr_staff_death['deaths_covid']=(corr_staff_death['critical_staffing_shortage_today_yes']-corr_staff_death['critical_staffing_shortage_today_yes'].min())/(corr_staff_death['critical_staffing_shortage_today_yes'].max()-corr_staff_death['critical_staffing_shortage_today_yes'].min())
corr_staff_death['deaths_covid']=(corr_staff_death['deaths_covid']-corr_staff_death['deaths_covid'].min())/(corr_staff_death['deaths_covid'].max()-corr_staff_death['deaths_covid'].min())

C:\Users\Julian\AppData\Local\Temp\ipykernel_1868\1872936165.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Julian\AppData\Local\Temp\ipykernel_1868\1872936165.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
corr_staff_death[['critical_staffing_shortage_today_yes','deaths_covid']].corr()

,critical_staffing_shortage_today_yes,deaths_covid
critical_staffing_shortage_today_yes,1.0,1.0
deaths_covid,1.0,1.0


8 - Siguiendo las respuestas anteriores, ¿cuál fue el peor mes de la pandemia para USA en su conjunto? Puede utilizar otras medidas que considere necesarias.

In [111]:
df_deaths = df[['date','deaths_covid']].sort_values('date', ascending=False)
df_deaths = df_deaths.groupby(pd.Grouper(key='date', axis=0,freq='M'))[['deaths_covid']].sum().sort_values('deaths_covid', ascending=False).reset_index().head(5)

df_beds = df[['date','inpatient_bed_covid_utilization_numerator']]
df_beds=df_beds.groupby(pd.Grouper(key='date', axis=0,freq='M'))[['inpatient_bed_covid_utilization_numerator']].sum().sort_values('inpatient_bed_covid_utilization_numerator', ascending=False).reset_index().head(5)
worst_month = pd.merge(df_deaths,df_beds,on='date',how='left')
worst_month.dropna()

,date,deaths_covid,inpatient_bed_covid_utilization_numerator
0,2021-01-31,66020.0,3911405.0
1,2020-12-31,56674.0,3701542.0
3,2021-09-30,44903.0,2778871.0
4,2022-01-31,42246.0,4478045.0


In [112]:
worst_month['deaths_covid']=round(worst_month['deaths_covid'],0)
worst_month

,date,deaths_covid,inpatient_bed_covid_utilization_numerator
0,2021-01-31,66020.0,3911405.0
1,2020-12-31,56674.0,3701542.0
2,2020-05-31,54884.0,NaN
3,2021-09-30,44903.0,2778871.0
4,2022-01-31,42246.0,4478045.0


In [100]:
df_beds = df[['date','inpatient_bed_covid_utilization_numerator','deaths_covid']]
#df_beds = df_beds.groupby(pd.Grouper(key='date', axis=0,freq='M'))[['inpatient_bed_covid_utilization_numerator']].sum().sort_values('inpatient_bed_covid_utilization_numerator', ascending=False)
#df_beds['inpatient_bed_covid_utilization_numerator']=round(df_beds['inpatient_bed_covid_utilization_numerator']/2.43,0)
df_beds = df_beds.groupby(pd.Grouper(key='date', axis=0,freq='M')).sum()
df_beds=df_beds.reset_index()



date            0
deaths_covid    0
dtype: int64

In [94]:
df_deaths

,date,deaths_covid
0,2021-01-31,66020.0
1,2020-12-31,56674.0
2,2020-05-31,54884.0
3,2021-09-30,44903.0
4,2022-01-31,42246.0


In [86]:
worst_month

,date,deaths_covid,inpatient_bed_covid_utilization_numerator
0,2021-01-31,66020.0,3911405.0
1,2020-12-31,56674.0,3701542.0
2,2020-05-31,54884.0,NaN
3,2021-09-30,44903.0,2778871.0
4,2022-01-31,42246.0,4478045.0


In [97]:
states=df_deaths['date'].head()

fig = go.Figure(data=[
    go.Bar(name='% máximo camas ICU por COVID', x=states, y=df_deaths['deaths_covid'])])
fig.update_layout(title='Cantidad de muertos por Estado')

fig.show()

9 - ¿Qué recomendaciones haría, ex post, con respecto a los recursos hospitalarios y su uso?

Prepararse para tener Staff medico de reserva, la falta de este tiene efectos directos en la cantidad de muertes.

Aunque en promedio la utilización de camas alcanzó, en los momentos de máxima demanda hubo faltante en Illinois y Washington

Calculadora de internaciones entre dos fechas

In [ ]:
init = st.date_input(
     "Seleccionar fecha de inicio",
     datetime.date(2019, 7, 6))
fin = st.date_input(
     "Seleccionar último dia de análisis",
     datetime.date(2019, 7, 6))


calc_inter = df_TotalPais_used_bed_covid[(df_TotalPais_used_bed_covid['date'].dt.date > pd.to_datetime(init)) & (df_TotalPais_used_bed_covid['date'].dt.date < pd.to_datetime(fin))].sort_values('date',ascending=True)

calc_inter = round(calc_inter['inpatient_bed_covid_utilization_numerator'].sum()/promedio_internacion,2)
st.write('La candidad de internaciones entre:', init, 'y',fin,'fue de ',calc_inter)
